# Experiment and Registry the best model

In [17]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error



In [18]:
import mlflow

mlflow.set_tracking_uri('sqlite:///mlflow.db')


In [19]:
mlflow.set_experiment("nyc-taxi-expertiment")

<Experiment: artifact_location='/workspaces/mlops/04- deployment/web-service-mlflow/mlruns/1', creation_time=1726098508429, experiment_id='1', last_update_time=1726098508429, lifecycle_stage='active', name='nyc-taxi-expertiment', tags={}>

To run: mlflow ui --port 8080 --backend-store-uri sqlite:///mlflow.db

# Getting the data

In [20]:
mlflow.set_experiment("nyc-taxi-expertiment")
experiment = mlflow.get_experiment_by_name("nyc-taxi-expertiment")

In [21]:
experiment

<Experiment: artifact_location='/workspaces/mlops/04- deployment/web-service-mlflow/mlruns/1', creation_time=1726098508429, experiment_id='1', last_update_time=1726098508429, lifecycle_stage='active', name='nyc-taxi-expertiment', tags={}>

In [22]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [23]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [24]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [25]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [26]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

# Hyperparameter tunning and tracking with Hyperopt and MLFlow

In [27]:
import xgboost as xgb
from mlflow import MlflowClient


In [28]:
pip freeze

aiohttp @ file:///tmp/build/80754af9/aiohttp_1646806365504/work
aiosignal @ file:///tmp/build/80754af9/aiosignal_1637843061372/work
alabaster @ file:///home/ktietz/src/ci/alabaster_1611921544520/work
alembic==1.13.2
anaconda-client @ file:///tmp/build/80754af9/anaconda-client_1635342557008/work
anaconda-navigator==2.1.4
anaconda-project @ file:///tmp/build/80754af9/anaconda-project_1637161053845/work
aniso8601==9.0.1
anyio @ file:///tmp/build/80754af9/anyio_1644463572971/work/dist
appdirs==1.4.4
argon2-cffi @ file:///opt/conda/conda-bld/argon2-cffi_1645000214183/work
argon2-cffi-bindings @ file:///tmp/build/80754af9/argon2-cffi-bindings_1644569679365/work
arrow @ file:///opt/conda/conda-bld/arrow_1649166651673/work
astroid @ file:///tmp/build/80754af9/astroid_1628063140030/work
astropy @ file:///opt/conda/conda-bld/astropy_1650891077797/work
asttokens @ file:///opt/conda/conda-bld/asttokens_1646925590279/work
async-timeout @ file:///tmp/build/80754af9/async-timeout_1637851218186/work
a

Note: you may need to restart the kernel to use updated packages.


In [29]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
from hyperopt.pyll import stochastic

In [30]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [31]:
#mlflow.xgboost.autolog(log_models=True)

def print_auto_logged_info(r):
    tags = {k: v for k, v in r.data.tags.items() if not k.startswith("mlflow.")}
    artifacts = [f.path for f in MlflowClient().list_artifacts(r.info.run_id, "model")]
    print(f"run_id: {r.info.run_id}")
    print(f"artifacts: {artifacts}")
    print(f"params: {r.data.params}")
    print(f"metrics: {r.data.metrics}")
    print(f"tags: {tags}")


In [32]:
def objective(params):
    with mlflow.start_run() as run:
        mlflow.set_tag("model", "xgboost")
        mlflow.set_tag('developer','Willian Lima')
        mlflow.log_param('train-path','https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
        mlflow.log_param('valid-path','https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        mlflow.xgboost.log_model(booster,artifact_path='model')
        #print_auto_logged_info(mlflow.get_run(run_id=run.info.run_id))
        with open('dict_vectorizor.bin', 'wb') as f_out:
            pickle.dump((dv, booster), f_out)
        mlflow.log_artifact('dict_vectorizor.bin')
    return {'loss': rmse, 'status': STATUS_OK}

hyperopt.fmin(). It covers how to write an objective function that fmin can optimize, and how to describe a search space that fmin can search.

Hyperopt's job is to find the best value of a scalar-valued, possibly-stochastic function over a set of possible arguments to that function. Whereas many optimization packages will assume that these inputs are drawn from a vector space, Hyperopt is different in that it encourages you to describe your search space in more detail. By providing more information about where your function is defined, and where you think the best values are, you allow algorithms in hyperopt to search more efficiently.

I'd recommend hyperopt instead of scikit-learn's GridSearchCV. Hyperopt can search the space with Bayesian optimization using hyperopt.tpe.suggest. It will arrive at good parameters faster than a grid search and you can limit the number of iterations no matter the space size, so it's definitely better for large spaces.

In [33]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=8,
    trials=Trials()
)

[01:31:09] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:17.68852                                                                                      
[1]	validation-rmse:14.95026                                                                                      
[2]	validation-rmse:12.83591                                                                                      
[3]	validation-rmse:11.21771                                                                                      
[4]	validation-rmse:10.00008                                                                                      
[5]	validation-rmse:9.08931                                                                                       
[6]	validation-rmse:8.42124                                                                                       
[7]	validation-rmse:7.93461                                                 

[70]	validation-rmse:6.51857                                                                                      
[71]	validation-rmse:6.51815                                                                                      
[72]	validation-rmse:6.51818                                                                                      
[73]	validation-rmse:6.51812                                                                                      
[74]	validation-rmse:6.51769                                                                                      
[75]	validation-rmse:6.51716                                                                                      
[76]	validation-rmse:6.51694                                                                                      
[77]	validation-rmse:6.51641                                                                                      
[78]	validation-rmse:6.51639                                                    

[141]	validation-rmse:6.50558                                                                                     
[142]	validation-rmse:6.50580                                                                                     
[143]	validation-rmse:6.50556                                                                                     
[144]	validation-rmse:6.50546                                                                                     
[145]	validation-rmse:6.50533                                                                                     
[146]	validation-rmse:6.50530                                                                                     
[147]	validation-rmse:6.50524                                                                                     
[148]	validation-rmse:6.50528                                                                                     
[149]	validation-rmse:6.50507                                                   

[212]	validation-rmse:6.50539                                                                                     
[213]	validation-rmse:6.50518                                                                                     
[214]	validation-rmse:6.50526                                                                                     
[215]	validation-rmse:6.50590                                                                                     
[216]	validation-rmse:6.50591                                                                                     
[217]	validation-rmse:6.50617                                                                                     
[218]	validation-rmse:6.50605                                                                                     
[219]	validation-rmse:6.50607                                                                                     
[220]	validation-rmse:6.50593                                                   

/home/codespace/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")

2024/09/12 01:32:18 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



...

2024/09/12 01:36:12 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



[01:36:12] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:8.83254                                                                                       
[1]	validation-rmse:6.95975                                                                                       
[2]	validation-rmse:6.66943                                                                                       
[3]	validation-rmse:6.60753                                                                                       
[4]	validation-rmse:6.58947                                                                                       
[5]	validation-rmse:6.57332                                                                                       
[6]	validation-rmse:6.56548                                                                                       
[7]	validation-rmse:6.56005                                                 

[70]	validation-rmse:6.50236                                                                                      
[71]	validation-rmse:6.50402                                                                                      
[72]	validation-rmse:6.50415                                                                                      
[73]	validation-rmse:6.50394                                                                                      
[74]	validation-rmse:6.50397                                                                                      
[75]	validation-rmse:6.50365                                                                                      
[76]	validation-rmse:6.50291                                                                                      
[77]	validation-rmse:6.50365                                                                                      
[78]	validation-rmse:6.50244                                                    

2024/09/12 01:36:27 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



100%|██████████████████████████████████████████████| 8/8 [05:18<00:00, 39.82s/trial, best loss: 6.360835859587613]


# Interacting with the MLflow tracking server
The MlflowClient object allows us to interact with...

an MLflow Tracking Server that creates and manages experiments and runs.
an MLflow Registry Server that creates and manages registered models and model versions.
To instantiate it we need to pass a tracking URI and/or a registry URI

In [34]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

Showing existing experiments

In [35]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

list_experiments = client.search_experiments()

for experiment in list_experiments:
    print(f"Name: {experiment.name}, Id: {experiment.experiment_id}")

Name: nyc-taxi-expertiment, Id: 1


Getting the top 5 better runs

In [36]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [37]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")
    #print(run)

run id: d716224711f645d184ef19b9e58cf7e0, rmse: 6.3608
run id: e26f28b913fc4e8190fc356d8ba61eb1, rmse: 6.3710
run id: 64b83260e4c4473d914426db74b3943f, rmse: 6.3830
run id: 0a1f2403ab4a4a199aee42428fa2c021, rmse: 6.4476
run id: ccc1fba45bc244cd8247db76fc430a1f, rmse: 6.4725


In [38]:
# get the best model ID
best_model_id = runs[0].info.run_id
best_model_rmse = runs[0].data.metrics['rmse']


# Interacting with the Model Registry
In this section We will use the MlflowClient instance to:

Register a new version for the best experiment nyc-taxi-regressor


In [39]:
# List all registered models
registered_models = client.search_registered_models()

# Print out model names
for model in registered_models:
    print(f"Model Name: {model.name}")
    
model_name = "nyc-taxi-regressor"

Model Name: nyc-taxi-regressor


In [40]:
# Register the model
model_uri = f"runs:/{best_model_id}/model"
model_info = mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")
# Get the version of the new best model
model_version = model_info.version

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '2' of model 'nyc-taxi-regressor'.


Verify if the RSME of the new model is better than the model already in the alias "champion"

In [41]:
try:
    # get a model version by alias
    run_id = client.get_model_version_by_alias(model_name, "champion").run_id
    run = client.get_run(run_id)
    experiment_id = run.info.experiment_id
    experiment = client.get_experiment(experiment_id)

    # Print out experiment details
    print(f"Experiment ID: {experiment.experiment_id}")
    print(f"Name: {experiment.name}")
    print(f"Artifact Location: {experiment.artifact_location}")
    print(f"Lifecycle Stage: {experiment.lifecycle_stage}")

    # Print out run details
    print(f"Run ID: {run.info.run_id}")
    print(f"Start Time: {run.info.start_time}")
    print(f"End Time: {run.info.end_time}")
    print(f"Status: {run.info.status}")
    print(f"Parameters: {run.data.params}")
    print(f"Metrics: {run.data.metrics}")
    print(f"Tags: {run.data.tags}")

    current_model_rsme = run.data.metrics['rmse']
except:
    print("There is no champion still!")

Experiment ID: 1
Name: nyc-taxi-expertiment
Artifact Location: /workspaces/mlops/04- deployment/web-service-mlflow/mlruns/1
Lifecycle Stage: active
Run ID: 64b83260e4c4473d914426db74b3943f
Start Time: 1726099579201
End Time: 1726099606732
Status: FINISHED
Parameters: {'train-path': 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet', 'valid-path': 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet', 'learning_rate': '0.518687637319942', 'max_depth': '66', 'min_child_weight': '1.2466226265419926', 'objective': 'reg:linear', 'reg_alpha': '0.2753327527265053', 'reg_lambda': '0.045202518934187176', 'seed': '42'}
Metrics: {'rmse': 6.382990187323998}
Tags: {'mlflow.user': 'codespace', 'mlflow.source.name': '/home/codespace/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py', 'mlflow.source.type': 'LOCAL', 'mlflow.runName': 'defiant-kite-111', 'model': 'xgboost', 'developer': 'Willian Lima', 'mlflow.log-model.history': '[{"run

In [42]:
try:
    current_model_version = client.get_model_version_by_alias(model_name, "champion").version
    # If the new model is better than the current champion, replace it
    if best_model_rmse <= current_model_rsme:
        # Set model version tag
        # Seting this new model to validation pending
        client.set_model_version_tag(model_name, model_version, "validation_status", "pending")
        # Seting a alias to this model champion to show that is the best from this experiment
        client.set_registered_model_alias(model_name, "champion", model_version)
        print(f"The new model is better than the current champion! Congratz!! Replacing the allias champion to the new model. Now, the version {model_version}")

    else:
        print(f"The new version os not better than the champion, keeping the current champion. The current champion is still {current_model_version}")
except:

    client.set_model_version_tag(model_name, model_version, "validation_status", "pending")
    # Seting a alias to this model champion to show that is the best from this experiment
    client.set_registered_model_alias(model_name, "champion", model_version)
    print(f"The new model is better than the current champion! Congratz!! Replacing the allias champion to the new model. Now, the version {model_version}")

The new model is better than the current champion! Congratz!! Replacing the allias champion to the new model. Now, the version 2


In [43]:
# Retrieving information about the model by alias

champion_model_version = client.get_model_version_by_alias(model_name, "champion").version

In [44]:
f"models:/{model_name}/{model_version}"

'models:/nyc-taxi-regressor/2'

Retriving the ''champion'' model and runing it

In [45]:
client.search_registered_models()


[<RegisteredModel: aliases={'champion': 2}, creation_timestamp=1726099958599, description=None, last_updated_timestamp=1726104987725, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1726104987725, current_stage='None', description=None, last_updated_timestamp=1726104987725, name='nyc-taxi-regressor', run_id='d716224711f645d184ef19b9e58cf7e0', run_link=None, source=('/workspaces/mlops/04- '
  'deployment/web-service-mlflow/mlruns/1/d716224711f645d184ef19b9e58cf7e0/artifacts/model'), status='READY', status_message=None, tags={'validation_status': 'pending'}, user_id=None, version=2>], name='nyc-taxi-regressor', tags={}>]

In [46]:
model_version

2

In [47]:
import mlflow.pyfunc

model_name = "nyc-taxi-regressor"
model_version = champion_model_version

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

model.predict(X_val)


[01:36:28] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


array([14.170389 ,  7.0993304, 16.036623 , ..., 13.533341 ,  6.614112 ,
        8.1268425], dtype=float32)

In [ ]:
run_id = client.get_model_version_by_alias(model_name, "champion").run_id


In [ ]:
path = client.download_artifacts(run_id=run_id, path='dict_vectorizor.bin')


In [ ]:
stdin = xgb.DMatrix(X_val)

In [ ]:
model.predict(stdin)